In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import gzip
import json
import pandas as pd
import numpy as np
import glob

# Step 1: Define Strata (States) and Sampling Parameters

DATA_PATH = '/content/drive/MyDrive/AFML_data/'

# 1. Define your 8 states and their full file paths
review_files_map = {
    DATA_PATH + 'review-Kentucky_10.json.gz': 'Kentucky',
    DATA_PATH + 'review-Hawaii_10.json.gz': 'Hawaii',
    DATA_PATH + 'review-Minnesota_10.json.gz': 'Minnesota',
    DATA_PATH + 'review-California_10.json.gz': 'California',
    DATA_PATH + 'review-Texas_10.json.gz': 'Texas',
    DATA_PATH + 'review-New_York_10.json.gz': 'NewYork',
    DATA_PATH + 'review-Florida_10.json.gz': 'Florida',
    DATA_PATH + 'review-Illinois_10.json.gz': 'Illinois'
}

# 2. INCREASED SAMPLING TARGETS
# We need to start with MANY users because most will be filtered out
# when we look for "Travelers" and "Popular Businesses".
TARGET_TRAIN_USERS_PER_STATE = 20000
TARGET_TEST_USERS_PER_STATE = 5000

# --- THIS IS THE UNZIPPING CODE ---
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield json.loads(l)
# ----------------------------------

# Step 2: Perform Stratified User Sampling

print("Starting Stage 1: Stratified User Selection")
train_user_set = set()
test_user_set = set()
state_user_map = {}

# 1. Loop 1: Find all users in each state (stratum)
for file_path, state_name in review_files_map.items():
    print(f"Loading users from: {file_path}...")
    user_set_for_state = set()
    try:
        for review in parse(file_path):
            if 'user_id' in review:
                user_set_for_state.add(review['user_id'])
    except FileNotFoundError:
        print(f"  WARNING: File not found: {file_path}. Skipping stratum.")
        continue

    state_user_map[state_name] = user_set_for_state
    print(f"  Found {len(user_set_for_state)} unique users in {state_name}.")

# 2. Loop 2: Sample disjoint train/test sets from each stratum
np.random.seed(42)
for state_name, user_set in state_user_map.items():

    user_list = list(user_set)
    np.random.shuffle(user_list)

    total_users_in_state = len(user_list)

    # 1. Take Train Users
    train_sample_size = min(TARGET_TRAIN_USERS_PER_STATE, total_users_in_state)
    sampled_train_users = user_list[:train_sample_size]
    train_user_set.update(sampled_train_users)

    # 2. Take Test Users from the *remainder*
    remaining_users = user_list[train_sample_size:]
    test_sample_size = min(TARGET_TEST_USERS_PER_STATE, len(remaining_users))

    if test_sample_size > 0:
        sampled_test_users = remaining_users[:test_sample_size]
        test_user_set.update(sampled_test_users)

print("\n--- Stage 1 Complete ---")
print(f"Total unique TRAIN users selected: {len(train_user_set)}")
print(f"Total unique TEST users selected: {len(test_user_set)}")

Starting Stage 1: Stratified User Selection
Loading users from: /content/drive/MyDrive/AFML_data/review-Kentucky_10.json.gz...
  Found 145516 unique users in Kentucky.
Loading users from: /content/drive/MyDrive/AFML_data/review-Hawaii_10.json.gz...
  Found 64336 unique users in Hawaii.
Loading users from: /content/drive/MyDrive/AFML_data/review-Minnesota_10.json.gz...
  Found 182652 unique users in Minnesota.
Loading users from: /content/drive/MyDrive/AFML_data/review-California_10.json.gz...
  Found 1378206 unique users in California.
Loading users from: /content/drive/MyDrive/AFML_data/review-Texas_10.json.gz...
  Found 1260383 unique users in Texas.
Loading users from: /content/drive/MyDrive/AFML_data/review-New_York_10.json.gz...
  Found 630637 unique users in NewYork.
Loading users from: /content/drive/MyDrive/AFML_data/review-Florida_10.json.gz...
  Found 1155514 unique users in Florida.
Loading users from: /content/drive/MyDrive/AFML_data/review-Illinois_10.json.gz...
  Found 43

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter # Import Counter

print("Starting Stage 2: Loading full review data for selected users...")
train_reviews_list = []
test_reviews_list = []

# Loop through all 8 files again to get full review data
for file_path, state_name in review_files_map.items():
    print(f"Parsing {file_path} for selected user data...")
    try:
        for review in parse(file_path):
            user_id = review.get('user_id')

            if user_id in train_user_set:
                review['state'] = state_name
                train_reviews_list.append(review)
            elif user_id in test_user_set:
                review['state'] = state_name
                test_reviews_list.append(review)

    except FileNotFoundError:
        print(f"  WARNING: File not found: {file_path}. Skipping.")
        continue

print("\nFinished parsing all files. Now filtering for popular businesses...")

# --- STEP 1: Filter for gmap_ids with > 30 reviews ---
MIN_REVIEWS_PER_BUSINESS = 30

all_reviews_list = train_reviews_list + test_reviews_list
print(f"Total reviews found (Train+Test): {len(all_reviews_list)}")

all_gmap_ids = [r['gmap_id'] for r in all_reviews_list if 'gmap_id' in r]
gmap_counts = Counter(all_gmap_ids)
print(f"Found {len(gmap_counts)} total unique businesses (gmap_ids).")

valid_gmap_ids = {gmap_id for gmap_id, count in gmap_counts.items() if count > MIN_REVIEWS_PER_BUSINESS}
print(f"Found {len(valid_gmap_ids)} businesses with > {MIN_REVIEWS_PER_BUSINESS} reviews.")

filtered_train_reviews_list = [r for r in train_reviews_list if r.get('gmap_id') in valid_gmap_ids]
filtered_test_reviews_list = [r for r in test_reviews_list if r.get('gmap_id') in valid_gmap_ids]

print(f"Train reviews reduced from {len(train_reviews_list)} to {len(filtered_train_reviews_list)}")
print(f"Test reviews reduced from {len(test_reviews_list)} to {len(filtered_test_reviews_list)}")
# --- End of STEP 1 ---


print("\nNow processing filtered datasets...")

def process_and_save_dataset(reviews_list, output_csv_name, id_offset=0):
    """
    Processes a list of reviews, applies gmap_id filter, then
    filters for users with 'is_diff_state=True', saves to CSV,
    and returns the final DataFrame AND the max ID used before user filtering.
    """
    print(f"\n--- Processing dataset for {output_csv_name} ---")
    if not reviews_list:
        print("No reviews found for this dataset. Skipping.")
        return None, id_offset # Return offset unchanged

    # Step 3: Create DataFrame and Clean
    print("Creating final DataFrame...")
    all_reviews_df = pd.DataFrame(reviews_list)

    columns_to_keep = ['user_id', 'rating', 'text', 'gmap_id', 'state', 'time']
    valid_columns = [col for col in columns_to_keep if col in all_reviews_df.columns]
    all_reviews_df = all_reviews_df[valid_columns]

    print("Cleaning data and converting datetime...")
    cleaned_df = all_reviews_df.dropna(subset=['user_id', 'text', 'rating', 'gmap_id'])

    cleaned_df = cleaned_df.copy()
    cleaned_df['datetime'] = pd.to_datetime(cleaned_df['time'], unit='ms')
    cleaned_df = cleaned_df.drop(columns=['time'])

    # Step 4: Create Timeline & Geodiversity Features
    print("\nSorting DataFrame by user and time...")
    df_sorted = cleaned_df.sort_values(by=['user_id', 'datetime'])

    print("Calculating time and state differences...")
    df_sorted['last_review_datetime'] = df_sorted.groupby('user_id')['datetime'].shift()
    df_sorted['last_review_state'] = df_sorted.groupby('user_id')['state'].shift()

    df_sorted['time_since_last_review_sec'] = (df_sorted['datetime'] - df_sorted['last_review_datetime']).dt.total_seconds()
    df_sorted['is_diff_state'] = (df_sorted['state'] != df_sorted['last_review_state'])

    df_sorted['time_since_last_review_sec'] = df_sorted['time_since_last_review_sec'].fillna(-1)
    df_sorted.loc[df_sorted['last_review_state'].isnull(), 'is_diff_state'] = False

    print("Creating final review_id column...")
    df_sorted.reset_index(drop=True, inplace=True)
    df_sorted['review_id'] = df_sorted.index + 1 + id_offset

    # Capture the max ID before we filter users
    rows_before_user_filter = len(df_sorted)

    # --- NEW STEP 2: Filter for users with at least one 'is_diff_state=True' ---
    print("\n--- Applying 'is_diff_state' user filter ---")
    true_count_before = df_sorted['is_diff_state'].sum()
    total_reviews_before = len(df_sorted)
    if total_reviews_before > 0:
        print(f"  Before filter: {true_count_before} 'True' reviews out of {total_reviews_before} total ({true_count_before/total_reviews_before:.2%})")
    else:
        print("  Before filter: 0 reviews.")

    users_with_diff_state_true = df_sorted[df_sorted['is_diff_state'] == True]['user_id'].unique()
    print(f"  Found {len(users_with_diff_state_true)} users with at least one 'is_diff_state=True' review.")

    if len(users_with_diff_state_true) == 0:
        print("  WARNING: No users found with 'is_diff_state=True'. The resulting file will be empty.")
        # Create an empty dataframe with the correct columns to avoid errors
        df_final_filtered = pd.DataFrame(columns=df_sorted.columns)
    else:
        print(f"  Filtering DataFrame to *only* include all reviews from these users.")
        df_final_filtered = df_sorted[df_sorted['user_id'].isin(users_with_diff_state_true)]

    true_count_after = df_final_filtered['is_diff_state'].sum()
    total_reviews_after = len(df_final_filtered)

    if total_reviews_after > 0:
        print(f"  After filter: {total_reviews_after} total reviews remaining.")
        print(f"  New balance: {true_count_after} 'True' reviews ({true_count_after/total_reviews_after:.2%})")
    else:
        print("  After filter: 0 total reviews remaining.")
    # --- End of NEW STEP 2 ---

    # Step 5: Save Your Final Enriched CSV
    final_columns_to_save = [
        'review_id',
        'user_id',
        'rating',
        'text',
        'gmap_id',
        'state', # <-- ADDED THIS COLUMN
        'datetime',
        'last_review_datetime',
        'time_since_last_review_sec',
        'is_diff_state'
    ]
    final_columns_to_save = [col for col in final_columns_to_save if col in df_final_filtered.columns]

    print(f"\nFinal columns will be: {final_columns_to_save}")
    # Use the filtered dataframe
    final_df_to_save = df_final_filtered[final_columns_to_save]

    # --- MODIFIED: New file name based on your targets ---
    final_csv_path = output_csv_name # Use the name directly
    print(f"\nSaving final stratified data to {final_csv_path}...")
    final_df_to_save.to_csv(final_csv_path, index=False)

    print(f"--- Processing Complete! ---")
    print(f"Your file '{final_csv_path}' is ready. (Total Reviews: {len(final_df_to_save)})\\n")

    # Return the filtered DF and the count *before* this filter for the offset
    return final_df_to_save, rows_before_user_filter

# --- End of function definition ---

# Now, process both FILTERED datasets sequentially

# 1. Process train first, starting with ID 1 (offset=0)
# *** UPDATED FILENAMES ***
train_df, last_train_id = process_and_save_dataset(
    filtered_train_reviews_list,
    'TRAIN_filtered_target_100k.csv',
    id_offset=0
)

# 2. Get the max ID from the train set *before* the user filter
if train_df is None:
    last_train_id = 0 # Handle case where train set might be empty

print(f"Train set complete. Max review_id assigned was: {last_train_id}")

# 3. Process test, starting IDs *after* the last train ID
print(f"Starting test set processing with review_id offset of: {last_train_id}")
# *** UPDATED FILENAMES ***
test_df, _ = process_and_save_dataset(
    filtered_test_reviews_list,
    'TEST_filtered_target_25k.csv',
    id_offset=last_train_id
)

print("--- All Done! Both filtered files have been created with globally unique review_ids. ---")

Starting Stage 2: Loading full review data for selected users...
Parsing /content/drive/MyDrive/AFML_data/review-Kentucky_10.json.gz for selected user data...


In [1]:
import pandas as pd
import numpy as np

# --- 1. SETTINGS ---
INPUT_FILE = 'TRAIN_final_100k_clustered.csv'
OUTPUT_FILE = 'TRAIN_model_ready_final_mode.csv'
print(f"Loading {INPUT_FILE}...")

df = pd.read_csv(INPUT_FILE, parse_dates=['datetime', 'last_review_datetime'])
print(f"Loaded {len(df)} reviews.")

# --- 2. Feature Engineering: Datetime ---
print("Extracting datetime features...")
df['review_hour'] = df['datetime'].dt.hour
df['review_age_days'] = (df['datetime'].max() - df['datetime']).dt.total_seconds() / (60*60*24)

# --- 3. Bucketing ---
print("Bucketing 'time_since_last_review_sec'...")
bins = [-np.inf, -1, 0, 14400, 86400, 604800, 2592000, np.inf]
labels = ['First_Review', 'Same_Time', 'Under_4_Hours', 'Under_1_Day', 'Under_1_Week', 'Under_30_Days', 'Over_30_Days']
df['time_since_last_bucket'] = pd.cut(df['time_since_last_review_sec'], bins=bins, labels=labels, right=True)

# --- 4. User Behavior Features (MODE) ---
print("Calculating user behavior features...")
def get_mode(x):
    if x.mode().empty: return np.nan
    return x.mode().iloc[0]

user_grouped = df.groupby('user_id')
user_stats = user_grouped.agg(
    user_review_count=('review_id', 'count'),
    user_mode_rating=('rating', get_mode),
    user_rating_variance=('rating', 'var'),
    user_gmap_diversity=('gmap_id', 'nunique'),
    user_state_diversity=('state', 'nunique')
).reset_index()

user_stats['user_rating_variance'] = user_stats['user_rating_variance'].fillna(0)
df = pd.merge(df, user_stats, on='user_id', how='left')

# --- 5. Business Features (MODE) ---
print("Calculating business features...")
business_grouped = df.groupby('gmap_id')
business_stats = business_grouped.agg(
    business_review_count=('review_id', 'count'),
    business_mode_rating=('rating', get_mode),
    business_rating_variance=('rating', 'var')
).reset_index()

business_stats['business_rating_variance'] = business_stats['business_rating_variance'].fillna(0)
df = pd.merge(df, business_stats, on='gmap_id', how='left')

# --- 6. Save ---

print(f"Saving to {OUTPUT_FILE}...")
# Drop raw columns we don't need for the model
final_df = df.drop(columns=['time_since_last_review_sec', 'last_review_datetime'])
final_df.to_csv(OUTPUT_FILE, index=False)

print(f"--- Complete! Saved {OUTPUT_FILE} ---")

Loading TRAIN_final_100k_clustered.csv...
Loaded 100008 reviews.
Extracting datetime features...
Bucketing 'time_since_last_review_sec'...
Calculating user behavior features...
Calculating business features...
Saving to TRAIN_model_ready_final_mode.csv...
--- Complete! Saved TRAIN_model_ready_final_mode.csv ---


In [ ]:
import pandas as pd
import numpy as np

# --- 1. SETTINGS ---
INPUT_FILE = 'TEST_final_25k_clustered.csv'  # <-- Use the test file
OUTPUT_FILE = 'TEST_model_ready_final_mode.csv' # <-- Create a new test file
print(f"Loading {INPUT_FILE}...")

# Load the dataset, making sure to parse datetime columns
df = pd.read_csv(
    INPUT_FILE,
    parse_dates=['datetime', 'last_review_datetime']
)
print(f"Loaded {len(df)} reviews.")


# --- 2. Feature Engineering: Datetime ---
print("Extracting datetime features...")
df['review_hour'] = df['datetime'].dt.hour
df['review_age_days'] = (df['datetime'].max() - df['datetime']).dt.total_seconds() / (60*60*24)


# --- 3. Feature Engineering: Bucketing 'time_since_last_review_sec' ---
print("Bucketing 'time_since_last_review_sec'...")
bins = [-np.inf, -1, 0, 14400, 86400, 604800, 2592000, np.inf]
labels = [
    'First_Review',
    'Same_Time',
    'Under_4_Hours',
    'Under_1_Day',
    'Under_1_Week',
    'Under_30_Days',
    'Over_30_Days'
]
df['time_since_last_bucket'] = pd.cut(df['time_since_last_review_sec'], bins=bins, labels=labels, right=True)


# --- 4. User Behavior Features (with MODE) ---
print("Calculating user behavior features...")

# Define a function to safely get the first mode
def get_mode(x):
    if x.mode().empty:
        return np.nan
    return x.mode().iloc[0]

user_grouped = df.groupby('user_id')

user_stats = user_grouped.agg(
    user_review_count=('review_id', 'count'),
    user_mode_rating=('rating', get_mode), # <-- CHANGED TO MODE
    user_rating_variance=('rating', 'var'),
    user_gmap_diversity=('gmap_id', 'nunique'),
    user_state_diversity=('state', 'nunique')
).reset_index()

user_stats['user_rating_variance'] = user_stats['user_rating_variance'].fillna(0)
df = pd.merge(df, user_stats, on='user_id', how='left')


# --- 5. Business Features (with MODE) ---
print("Calculating business features...")
business_grouped = df.groupby('gmap_id')

business_stats = business_grouped.agg(
    business_review_count=('review_id', 'count'),
    business_mode_rating=('rating', get_mode), # <-- CHANGED TO MODE
    business_rating_variance=('rating', 'var')
).reset_index()

business_stats['business_rating_variance'] = business_stats['business_rating_variance'].fillna(0)
df = pd.merge(df, business_stats, on='gmap_id', how='left')


# --- 6. Save Final File ---
print(f"All features created. Cleaning up redundant columns...")

# Drop the raw columns that are no longer needed
final_df = df.drop(columns=[
    'time_since_last_review_sec', # Replaced by the bucket
    'last_review_datetime'        # No longer needed by any model
])

print(f"Saving to {OUTPUT_FILE}...")
final_df.to_csv(OUTPUT_FILE, index=False)

print("\n--- Preprocessing Complete! ---")
print(f"Your file '{OUTPUT_FILE}' is ready for modeling.")
print("\nFinal Columns:")
print(final_df.info())

Loading TEST_final_25k_clustered.csv...
Loaded 25002 reviews.
Extracting datetime features...
Bucketing 'time_since_last_review_sec'...
Calculating user behavior features...
Calculating business features...
All features created. Cleaning up redundant columns...
Saving to TEST_model_ready_final_mode.csv...

--- Preprocessing Complete! ---
Your file 'TEST_model_ready_final_mode.csv' is ready for modeling.

Final Columns:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25002 entries, 0 to 25001
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   review_id                 25002 non-null  int64         
 1   user_id                   25002 non-null  object        
 2   rating                    25002 non-null  int64         
 3   text                      25002 non-null  object        
 4   gmap_id                   25002 non-null  object        
 5   state                     250